# OpenAI Assistant Functions
#### The purpose of this notebook is to store the code required to build assistants, threads, and the basis for communicating with them

In [1]:
# import statement and loading environment variables
import os
import pathlib
from google.cloud import storage 
import json
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()

### Create new Assistant and corresponding Thread

In [2]:
# Create new assistant and thread
import os
import pathlib
from google.cloud import storage 
import json
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()


def create_assistant(instructions, name, model, tools):
    my_assistant = openai.beta.assistants.create(
        instructions=instructions,
        name=name,
        model=model,
        tools=tools
    )

    thread = client.beta.threads.create()

    # Extracting the IDs and converting them to strings
    assistant_id = str(my_assistant.id)
    thread_id = str(thread.id)

    return assistant_id, thread_id

name = "Wilke"
model = "gpt-4"
tools = [{"type": "code_interpreter"}]
instructions = """
Create a function that takes a list of numbers and returns the sum of the list.
"""

assistant_id, thread_id = create_assistant(instructions, name, model, tools)

print("Assistant ID:", assistant_id)
print("Thread ID:", thread_id)


Assistant ID: asst_ZueQgY8vYh9grvdhTSe40lmV
Thread ID: thread_tCswfJaLOcZ0vPE6kTYPjilC


### Create Standalone Thread

In [3]:
new_thread = client.beta.threads.create()
print("New Thread ID:", new_thread.id)

New Thread ID: thread_0xv4erIG7utTkd4dFtgHZRZN


### Send Message to the assistant

In [9]:
import os
from dotenv import load_dotenv
import openai
from openai import OpenAI
import time


load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()



def send_message_to_assistant(thread_id, assistant_id, message, wait_time=5):
    # Send the message to the specified thread
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=message
    )

    # Create a run for the assistant to process the message
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )

    # Wait for a short period to allow the assistant to respond
    time.sleep(wait_time)

    # Retrieve the messages after the wait time
    messages = client.beta.threads.messages.list(thread_id)

    # Find the latest assistant's response in the thread
    for msg in reversed(messages.data):
        if msg.role == 'assistant':
            return msg.content[0].text.value

    return "No response from the assistant."

# Example usage
message = "Hello, Assistant! What is the square root of 15333344?"

response = send_message_to_assistant(thread_id, assistant_id, message)
print("Assistant's response:", response)


Assistant's response: The square root of 1544 is approximately 39.29.


### List all the messages from a thread

In [8]:
import os
from dotenv import load_dotenv
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.Client()


def print_all_messages_in_thread(thread_id):
    # Retrieve the list of messages in the thread
    response_messages = client.beta.threads.messages.list(thread_id)

    # Iterate through the messages and print them
    for msg in response_messages:
        print("Message ID:", msg.id)
        print("Role:", msg.role)
        print("Content:", msg.content)
        print("Created At:", msg.created_at)
        print("-------------------------")

# # Example usage
# thread_id = "thread_7240Vt96XgXA1K50RJ8OTXNY"
print_all_messages_in_thread(thread_id)


Message ID: msg_cRWumdM7lcAWWkBE9qmXu1RF
Role: assistant
Content: [MessageContentText(text=Text(annotations=[], value='The square root of 15333344 is approximately 3915.78.'), type='text')]
Created At: 1702483269
-------------------------
Message ID: msg_8rAhJrpvkjtF9dhKhkWchEJc
Role: user
Content: [MessageContentText(text=Text(annotations=[], value='Hello, Assistant! What is the square root of 15333344?'), type='text')]
Created At: 1702483264
-------------------------
Message ID: msg_K9iFN5GJ4pfawAFi8a95dgVS
Role: assistant
Content: [MessageContentText(text=Text(annotations=[], value='The square root of 15344 is approximately 123.87.'), type='text')]
Created At: 1702483249
-------------------------
Message ID: msg_JOI13pZu1O2mQAQfL7VtdMd2
Role: user
Content: [MessageContentText(text=Text(annotations=[], value='Hello, Assistant! What is the square root of 15344?'), type='text')]
Created At: 1702483245
-------------------------
Message ID: msg_Z3onq6NlEkQh7f0IR1NStcq0
Role: assistant
Co